# Model Server

Please run the following installs once:

In [1]:
# !pip install -U kfserving==0.2.0 lightgbm 
# !pip install -U azure
# !pip uninstall -y mlrun
# !pip install -U git+https://github.com/mlrun/mlrun.git#development

## nuclio code section

In [2]:
# nuclio: ignore
# If you have never run nuclio functions in your notebooks, please uncomment and run the following:
# !pip install nuclio-jupyter
import nuclio

Install the following packages available to the function:

In [3]:
%%nuclio cmd -c
pip install kfserving==0.2.0 --upgrade
pip install azure
pip install numpy lightgbm
pip install -U joblib

In [4]:
import kfserving
import os
import numpy as np
import joblib
from typing import List
import lightgbm as lgb

In [5]:
BOOSTER_FILE = "lightgbm.model.pickle"

class LGBoostModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, booster: lgb.LGBMClassifier = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if booster:
            self.ready = True

    def load(self):
        """Load model from Kubeflow storage.
        """
#        model_file = f'/User/projects/lightgbm/models/{BOOSTER_FILE}'  # for debugging
        model_file = os.path.join(
            kfserving.Storage.download(self.model_dir), BOOSTER_FILE)

        self.classifier = joblib.load(model_file)

    def predict(self, body: List) -> List:
        """Generate model predictions from sample.
        
        :param body : A list of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body)
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [6]:
# nuclio: end-code

In [7]:
from mlrun import new_model_server

In [8]:
fn = new_model_server('lgbm-server', 
                      models={'lgbm_v5_joblib': '/User/projects/lightgbm/models'}, 
                      model_class='LGBoostModel')
fn.with_v3io('User','~/')

In [9]:
#fn.verbose=True
addr = fn.deploy(project='github-demos')

[mlrun] 2019-12-18 20:44:45,302 deploy started
[nuclio.deploy] 2019-12-18 20:44:46,390 (info) Building processor image


INFO:nuclio.deploy:(info) Building processor image


[nuclio.deploy] 2019-12-18 20:44:52,446 (info) Build complete


INFO:nuclio.deploy:(info) Build complete


[nuclio.deploy] 2019-12-18 20:44:57,516 (info) Function deploy complete


INFO:nuclio.deploy:(info) Function deploy complete


[nuclio.deploy] 2019-12-18 20:44:57,522 done updating lgbm-server, function address: 3.136.3.102:30528


INFO:nuclio.deploy:done updating lgbm-server, function address: 3.136.3.102:30528


### Test data for events

In [10]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

import requests

In [11]:
# Grab some data from the test set
features = pq.read_table('/User/projects/lightgbm/data/raw/xtest').to_pandas()
labels = pq.read_table('/User/projects/lightgbm/data/raw/ytest').to_pandas()

In [12]:
# Seldon protocol event
event_seldon = {
    "data" : {
        "ndarray": features.values.tolist()
    }
}
csel = str(event_seldon).replace("\'", "\"").replace('\n', '').replace(' ', '')

In [13]:
resp = requests.put(addr + '/predict/lgbm_v5_joblib', data=csel)
print(resp.content)

b'{"data": {"ndarray": [1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0

### Test of Estimated Model Object 

Here we simply grab the estimated model file created on the [kubeflow pipeline](kubeflow%20pipeline.ipynb), load it, and run a test matrix through it.

In [ ]:
import joblib
model_file = '/User/projects/lightgbm/models/lightgbm.model.pickle'
lgbm_model = joblib.load(open(model_file, 'rb'))

#### test one row

In [49]:
testvals = [1.86888885,1.57777536,-0.39419246,2.05800271,-0.75799805,1.57381201,1.4090718,1.26302791,1.08653808,1.22390795,0.46826237,0.78775787,0.,0.84368491,0.33878541,-0.4717221,2.54822445,0.38823441,-0.57677436,1.70674813,0.,0.8656919,0.87541604,0.98195601,2.04881859,1.63834357,1.42886198,1.33937621]
testvals = np.asarray(testvals).reshape(-1,1).T

In [50]:
lgbm_model.predict(testvals)

array([0.])

#### test a matrix

In [51]:
lgbm_model.predict(features)

array([0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
       1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1.,
       1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
       1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0.])